In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing

In [2]:
df=pd.read_csv("E:\git_exercise\Химический анализ родника в Нескучном саду.csv", sep=';')
#Добавляем промежуточные столбцы
df['Не более']=''
df['В пределах - начало']=''
df['В пределах - конец']=''
df['Результат']=''
#Преобразуем результаты анализа во float, где это возможно 
for i in range(0,len(df)):
    df['Норматив'][i]=df['Норматив'][i].replace(',', '.')
    try:
        df['Результат анализа'][i] = float(df['Результат анализа'][i]) #.astype(float)[i]    
    except ValueError:
        df['Результат анализа'][i] = df['Результат анализа'][i]   
#Предварительный вывод df
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив,Не более,В пределах - начало,В пределах - конец,Результат
0,pH,единицы pH,8.4,в пределах 6-9,,,,
1,Запах,баллы,1,не более 2-3,,,,
2,Цветность,градусы,б/цвета,не более 30,,,,
3,Жёсткость,мг-эквл/дм3,9.2,в пределах 7-10,,,,
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,,,,
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,,,,
6,Нитраты (по NO3),мг/дм3,24,не более 45,,,,
7,Фосфаты (P),мг/дм3,0.36,не более 3.5,,,,
8,Хлориды (Cl),мг/дм3,200,не более 350,,,,
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,,,,


In [7]:
#Обработка данных - убираем фразы "не более","в пределах" и "-". Выносим пределы диапазонов в нужные столбцы
for i in range(0,len(df)):
    if df['Норматив'][i].find('не более')==0:
        cutstr1=df['Норматив'][i].replace('не более', '')
        df['Не более'][i]=float(cutstr1[cutstr1.find("-") + 1 : ])
    if df['Норматив'][i].find('в пределах')==0:
        cutstr2=df['Норматив'][i].replace('в пределах', '') 
        df['В пределах - начало'][i]=int(cutstr2[ : cutstr2.find("-")])
        df['В пределах - конец'][i]=int(cutstr2[cutstr2.find("-")+1 : ])
#Заполнение столбца Результат
for i in range(0,len(df)):
    if type(df['Результат анализа'][i])==str:
        df['Результат'][i]="Результат анализа не числовой"
    else:
        if df['Не более'][i]!='':
            if float(df['Результат анализа'][i])>float(df['Не более'][i]):
                df['Результат'][i]="Не норма" #str(df['Показатель'][i])+'>'+str(df['Не более'][i])
            else:
                df['Результат'][i]="Норма" #str(df['Показатель'][i])+'<='+str(df['Не более'][i])  
        else:
           # if type(df['Результат анализа'][i])!=str and float(df['Результат анализа'][i])>=float(df['В пределах - начало'][i]) and float(df['Результат анализа'][i])<=float(df['В пределах - конец'][i]):
            if float(df['Результат анализа'][i])>=float(df['В пределах - начало'][i]) and float(df['Результат анализа'][i])<=float(df['В пределах - конец'][i]):
                df['Результат'][i]="Норма" #str(df['В пределах - начало'][i])+'<='+str(df['Показатель'][i])+'<='+str(df['В пределах - конец'][i])
            else:
                df['Результат'][i]=df['Показатель'][i]+' выходит за пределы'
#df1 - это изначальный df без дополнительных столбцов
df1=df[['Показатель','Единица измерений','Результат анализа','Норматив','Результат']]
#Выводим со столбцом Показатель в качестве индекса
display(df1.set_index('Показатель'))

,Единица измерений,Результат анализа,Норматив,Результат
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Норма
Запах,баллы,1,не более 2-3,Норма
Цветность,градусы,б/цвета,не более 30,Результат анализа не числовой
Жёсткость,мг-эквл/дм3,9.2,в пределах 7-10,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,не более 1.5,Норма
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Норма
Нитраты (по NO3),мг/дм3,24,не более 45,Норма
Фосфаты (P),мг/дм3,0.36,не более 3.5,Норма
Хлориды (Cl),мг/дм3,200,не более 350,Норма
